<a href="https://colab.research.google.com/github/udayanand22/Lanchain/blob/main/narad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-cohere cohere transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [2]:
!pip install langchain_community


In [ ]:
import cohere
from langchain_cohere import ChatCohere
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableSequence

# Initialize Cohere
cohere_api_key = "nWna0uV7o3mC4IsL9WFejBV4wOf8BgB1YDB0Ew8D"  # Replace with your key
llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r")

# Hugging Face Summarizer and QA
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Setup memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Prompt with memory placeholder
prompt = PromptTemplate.from_template(
    """You are Narada, the celestial sage, known for your dramatic flair and poetic storytelling.
{chat_history}
User: Tell me a story about {topic}
Narada:"""
)

# Chain with memory
story_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

print("🦚 Narada Cohere Edition Activated — Now With Memory!")

while True:
    topic = input("🎤 Ask Narada for a story (or type 'exit'): ")
    if topic.lower() == 'exit':
        print("Goodbye! Narada's tales will await you next time.")
        break

    # Generate raw story with memory context
    raw_story = story_chain.run({"topic": topic})

    if raw_story.strip() == "":
        print("\n⚠️ Narada was unable to weave a tale. Please try another topic.")
        continue

    print("\n📜 Raw Narada Tale:\n", raw_story)

    try:
        summary = summarizer(raw_story, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
        print("\n🪶 Narada's Summary:\n", summary)

        # 👇 Memory print here
        print("\n🧠 Narada's Memory So Far:\n")
        for msg in memory.chat_memory.messages:
            print(f"{msg.type.title()}: {msg.content}")

        # QA relevance check
        result = qa_pipeline(question=topic, context=summary)
        score = result['score']

        if score < 0.5:
            print("\n⚠️ The answer seems to not align well with the question. Modifying the response...\n")
            modified_summary = f"Here's a more detailed answer about {topic}: {raw_story}"
            print("\n🪶 Modified Narada's Summary:\n", modified_summary)
        else:
            print("\n✅ Narada's Summary is aligned with the question.")

    except Exception as e:
        print(f"\n⚠️ An error occurred while summarizing or comparing: {e}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]